Reference: https://pennylane.ai/qml/demos/tutorial_mcm_introduction  

We will learn about  
* basic measurement processes in quantum mechanics,
* the impact of a measurement on one- and two-qubit systems,
* postselection and qubit reset, and
* dynamic quantum circuits powered by conditional operations.

## Measurements in quantum mechanics

### Mathematical description  
A measurement $M$ is a process that maps a valid quantum state $\rho$ to a classical probabilistic mixture  
$$M[\rho]=\sum_{i=1}^{n}{p_i\rho_i}$$
of post-measurement quantum states $\rho_i$ that are specified by $M$. Here, $n$ is the number of possible measurement outcomes and $p_i$ is the probability to measure the outcome $i$ associated to $\rho_i$, given the input state $\rho$. For a qubit in the $\vert+\rangle=(\vert 0\rangle + \vert 1 \rangle)/\sqrt{2}$ state measured in the $Z$ basis, we find  
$$M[\vert+\rangle\langle+\vert] = \frac{1}{2}\vert0\rangle\langle0\vert+\frac{1}{2}\vert1\rangle\langle1\vert$$
The expression above describes the probabilistic mixture after the quantum mechanical measurement if we do not record the measurement outcome. If we do record the measurement outcome and only keep those samples that match a specific postselection rule, we no longer have a probabilistic mixture, but find the state $\rho_i$ for the filtered outcome $i$.  

For the rest of this tutorial, we will restrict ourselves to standard measurements commonly found in mid-circuit measurements, using so-called projective measruements. In this setting, the measurement comes with one projector $\Pi_i$ per measurement outcome, and all projectors sum to the identity. The post-measurement states are given by
$$\rho_i = \frac{\Pi_i \rho \Pi_i}{tr[\Pi_i \rho]}$$
and the probabilities are dictated by the Born rule, $p_i = tr[\Pi_i \rho]$. This means that if we do not record the measurement outcome, the system simply ends up in the state  
$$M[\rho] =\sum_{i=1}^{n}{\Pi_i \rho \Pi_i}$$

### Measuring a single qubit  
consider a single qubit in the state $\vert +\rangle$. Let's implement this state in PennyLane and compute some expectaion values

In [2]:
import pennylane as qml

dev = qml.device('default.qubit')

@qml.qnode(dev)
def before():
    qml.H(0)
    return qml.expval(qml.Z(0)), qml.expval(qml.X(0))
b = before()
print('Z expectation before measurement:', b[0])
print('X expectation before measurement:', b[1])

Z expectation before measurement: 0.0
X expectation before measurement: 0.9999999999999996


Now we bring a mid-circuit measurement in the computational, or $Z$ basis. It comes with the projections $\Pi_i =\vert i\rangle\langle i\vert, i\in{0,1}$, onto the computational basis states. If we execute the measurement process but do not record the outcomes, we find the state  
$$M[\rho] = \Pi_0 \rho \Pi_0 + \Pi_1 \rho \Pi_1 = \frac{1}{2}I$$
This means that the measurement sneds the qubit from a pure state into a mixed state, i.e., it not only affects the state but even the class of states it is in. And this is because we did not even record the measurement outcome!  

Let's look at this example in PennyLane. We repaet the steps from above but additionally include a mid-circuit measurement calling ```measure()``` on the qubit ```0```.

In [5]:
@qml.qnode(dev)
def after():
    qml.H(0)
    qml.measure(0)
    return qml.expval(qml.Z(0)), qml.expval(qml.X(0))

a = after()
print('Z expectation after measurement:', a[0])
print('X expectation after measurement:', a[1])

Z expectation after measurement: 0.0
X expectation after measurement: 0.0


This measurement moved the qubit from the $\vert+\rangle$ eigenstate of the Pauli-$X$ operator into a mixed state with expectation value zero for all Pauli Operators, explaning the values we just observed.  

Now if we filter for one measurement outcome, say 0, we find the state  
$$M[\rho] = \rho_0 = \frac{\vert 0 \rangle \langle 0 \vert + \rangle \langle + \vert 0 \rangle \langle 0 \vert }{tr[\vert 0 \rangle \langle 0 \vert + \rangle \langle + \vert]} = \vert 0 \rangle \langle 0 \vert$$
that is , the qubit is a new, pure state. In PennyLane, we can postselection on the case where we measured a 0 using the ```postselect``` keyword argument of ```qml.measure```:

In [8]:
@qml.qnode(dev)
def after():
    qml.Hadamard(0)  # Create |+> state
    qml.measure(0, postselect=0)  # Measure and only accept 0 as outcome
    return qml.expval(qml.Z(0)), qml.expval(qml.X(0))

a = after()
print('Z expectation after measurement:', a[0])
print('X expectation after measurement:', a[1])

Z expectation after measurement: 1.0
X expectation after measurement: 0.0


As expected, we find that the measured, postselected qubit is in the $\vert 0 \rangle$ eigenstate of the Pauli-$Z$ operator with eigenvalue +1, yielding $\langle X \rangle = 0$ and $\langle Z \rangle = 1$

### Measuring a Bell pair  
Next, we consider a pair of qubits, entangled in a Bell state:  
$$\vert \phi \rangle = \frac{1}{\sqrt{2}}(\vert 00\rangle + \vert 11\rangle )$$
This is a pure state with density matrix  
$$\vert \phi\rangle\langle\phi\vert = \frac{1}{2}(\vert 00\rangle\langle00\vert + \vert00\rangle\langle11\vert + \vert11\rangle\langle00\vert +\vert11\rangle\langle11\vert)$$
We will again measure only the first qubit.

In [9]:
def bell_pari_preparation(**kwargs):
    qml.H(0)
    qml.CNOT(wires = [0,1])
    qml.measure(0, **kwargs)

Without recording the outcome, we obtain  
$$M[\rho]=\frac{1}{2}(\vert 00\rangle\langle00\vert+\vert11\rangle\langle11\vert)$$
which again could be described by a classical mixture as well. If we instead postselect on measuring, say, a 1, we find $M[\rho]=\vert11\rangle\langle11\vert$  

There are two striking differences between whether we record the measurement outcome or not: the state of the qubits changes from a mixed to a pure state, as witnessed by the state’s purity; and its entanglement changes, too, as witnessed by the von Neumann entanglement entropy. We can compute both quantities easily in PennyLane, using ```purity()``` and ```vn_entropy()```, respectively.

In [10]:
@qml.qnode(dev)
def bell_pair(postselect):
    bell_pari_preparation(postselect=postselect)
    return qml.purity([0,1]), qml.vn_entropy(0)

In [11]:
without_ps = bell_pair(None)
with_ps = bell_pair(1)
print(f"                     | without ps | with ps ")
print(f"Purity               |     {without_ps[0]:.1f}    |   {with_ps[0]:.1f}")
print(f"Entanglement entropy |     {without_ps[1]:.2f}   |  {with_ps[1]:.1f}")

                     | without ps | with ps 
Purity               |     0.5    |   1.0
Entanglement entropy |     0.69   |  -0.0


### Qubit reset  
Another commonly used feature with mid-circuit measurement is to reset the measured qubit, i.e., if we measured 1, we flip it back into to the $\vert 0\rangle$ state with a Pauli $X$ operation. f there is just one qubit, this is the same as if we never measured it but reset it directly to the initial state $\vert0\rangle$, as long as we do not use the measurement outcome for anything. For the Bell pari example from above, resetting the measured qubit means that we flip the first bit if it is a 1 . Alternatively, we can trace out the first qubit and re-initialize it in the state $\vert0\rangle$. Denoting the reset step explicitly as $R$, this leads to the post-measurement state  
\begin{align}
    R[M[\rho]] &= \vert0\rangle0\vert \otimes tr_1[M[\rho]] \\
            &= \vert0\rangle0\vert \otimes [\frac{1}{2} (\vert 0\rangle\langle0\vert  +\vert1\rangle\langle1\vert)] \\
            &= \vert0\rangle\langle0\vert\otimes \frac{1}{2}I
\end{align}
We see that the qubits are no longer entangled, even if we do not postselect. Let’s compute some exemplary expectation values in this state with PennyLane. We recycle the state preparation subroutine from above, to which we can simply pass the keyword argument '''reset''' to activate the qubit reset:

In [14]:
@qml.qnode(dev)
def bell_pair_with_reset(reset):
    bell_pari_preparation(reset=reset)
    return qml.expval(qml.Z(0)), qml.expval(qml.Z(1)), qml.expval(qml.Z(0) @ qml.Z(1))

no_reset = bell_pair_with_reset(reset=False)
reset = bell_pair_with_reset(reset=True)

print(f"              | <Z₀> | <Z₁> | <Z₀Z₁> ")
print(f"Without reset |  {no_reset[0]:.1f} |  {no_reset[1]:.1f} |   {no_reset[2]:.1f}")
print(f"With reset    |  {reset[0]:.1f} |  {reset[1]:.1f} |   {reset[2]:.1f}")

              | <Z₀> | <Z₁> | <Z₀Z₁> 
Without reset |  0.0 |  0.0 |   1.0
With reset    |  1.0 |  0.0 |   0.0


## Dynamically controlling a quantum circuit  
o far we’ve only talked about mid-circuit measurements that directly affect the state of qubits, about postselection, and about qubit reset as an additional step after performing the measurement. However, the outcomes of a measurement can not only be used to decide whether or not to discard a circuit execution. More importantly, as mid-circuit measurements are performed while the quantum circuit is up and running, their outcomes can be used to modify the circuit structure itself dynamically.

This technique is widely used to improve quantum algorithms or to trade off classical and quantum computing resources. It also is an elementary building block for quantum error correction, as the corrections need to happen while the circuit is running.

Here we look at a simple yet instructive example subroutine called a T-gadget, a technique related to quantum teleportation.



### T-gadget in PennyLane  
Applying a T gate on an error-corrected quantum computer is usually hard. A T-gadget [6] allows us to replace a T gate by Clifford gates, provided we have an auxiliary qubit in the right initial state, a so-called magic state. The gadget then consists of the following steps:  
* Prepare an auxiliary qubit in a magic state
* Entangle the auxiliary and target qubit with a ```CNOT```
* Measure the auxiliary qubit with ```measure`` and record the outcome
* Apply ```S``` conditionally

In [15]:
import numpy as np
magic_state = np.array([1, np.exp(1j * np.pi/4)])/np.sqrt(2)

def t_gadget(wires, aux_wires):
    qml.StatePrep(magic_state, wires =aux_wires)
    qml.CNOT(wires = [wires, aux_wires])
    mcm = qml.measure(aux_wires, reset = True)
    qml.cond(mcm, qml.S)(wires)

We wil not only derive why this works, but illustrate that this gadget implements a T gate by combining it with an adjoint $T^{\dagger}$ gate and looking at the resulting action on the eigenstate of ```X```. For this, we
* prepare plus and minus state
* apply T-gadget
* apply $T^{\dagger}$
* return $\langle X_0\rangle$

In [21]:
@qml.qnode(dev)
def test_t_gadget(init_state):
    qml.H(0) # Create |+> state
    if init_state == '-':
        qml.Z(0) # apply Z gate to get |->
    
    t_gadget(0, 1)
    qml.adjoint(qml.T)(0)

    return qml.expval(qml.X(0))

print(f"<X₀> with initial state |+>: {test_t_gadget('+'):4.1f}")
print(f"<X₀> with initial state |->: {test_t_gadget('-'):4.1f}")

<X₀> with initial state |+>:  1.0
<X₀> with initial state |->: -1.0


The T-gadget indeed performs a T gate, which is being reversed by T†. As a result, the expectation values match those of the initial states $\vert +-\rangle$.  

How can we understand the above circuit intuitively? We did not postselect the measurement outcome, but we did record (and use) it to modify the circuit structure. For a single measurement, or shot, this would have led to exactly one of the events “measure 0, do not apply ```S```" or "measure 1, apply ```S```", with equal probability for either one. The state on wire ```0``` is $T\vert +-\rangle$ in either case!